In [19]:
import pandas as pd
from urllib.parse import urlparse, unquote
import re

# Load dataset
df = pd.read_csv('data/csic_database.csv')

# Print columns for debugging
print("Original columns:", df.columns.tolist())

# Step 0: Calculate missing value ratio and drop columns > 70% missing (do this first)
missing_ratio = df.isna().mean()
threshold = 0.7
columns_to_drop = missing_ratio[missing_ratio > threshold].index
df = df.drop(columns=columns_to_drop)
print(f"Dropped columns with >{threshold*100}% missing values: {list(columns_to_drop)}")

# Step 1: Remove Unnamed Column
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Step 2: Determine normal host (after ensuring 'URL' exists)
def get_host(url):
    return urlparse(str(url)).netloc
if 'URL' in df.columns:
    normal_host = df['URL'].apply(get_host).mode()[0]
else:
    normal_host = 'localhost:8080'  # Fallback if 'URL' missing
    print("Warning: 'URL' column not found, using fallback host.")

# Step 3: Fix Spelling (check for 'contentLength' or 'lenght')
if 'lenght' in df.columns:
    df = df.rename(columns={'lenght': 'length'})
elif 'contentLength' in df.columns:
    df = df.rename(columns={'contentLength': 'length'})
else:
    df['length'] = 0  # Create if missing
    print("Warning: No 'length'-like column found, created with default 0.")

# Step 4: Standardize 'connection' (only if it exists)
if 'connection' in df.columns:
    df['connection'] = df['connection'].str.replace('Connection: close', 'close', case=False).str.strip()
else:
    print("Warning: 'connection' column not found, skipping standardization.")

# Step 5: Decode encoded data (URL and content-like column if present)
if 'URL' in df.columns:
    df['URL'] = df['URL'].apply(lambda x: unquote(str(x)) if pd.notna(x) else x)
content_col = next((col for col in ['content', 'payload', 'body'] if col in df.columns), None)
if content_col:
    df[content_col] = df[content_col].apply(lambda x: unquote(str(x)) if pd.notna(x) else x)
else:
    print("Warning: No content-like column (content, payload, body) found, skipping decoding.")

# Step 6: Flag anomalies
def categorize_anomaly(row, normal_host):
    label_col = 'label' if 'label' in df.columns else 'classification'
    if label_col in df.columns and row[label_col] in [0, 'Normal']:
        return 'None'
    url = str(row['URL']).lower() if 'URL' in df.columns else ''
    host = get_host(row['URL']) if 'URL' in df.columns else normal_host
    content = str(row[content_col]).lower() if content_col else ''
    if '<script' in content or '<script' in url:
        return 'XSS'
    elif "'" in content or '"' in content or 'union' in content or 'select' in content:
        return 'SQLi'
    elif '.bak' in url or '.inc' in url or '..' in url:
        return 'Path Traversal'
    elif row.get('Method', '') == 'PUT' or host != normal_host:
        return 'Suspicious Method/Host'
    return 'Other'

df['anomaly_type'] = df.apply(categorize_anomaly, axis=1, args=(normal_host,))

# Step 7: Normalize remaining missing values
df = df.fillna('')

# Step 8: Remove duplicates
df = df.drop_duplicates(keep='first')

# Step 9: Validate data types
df['length'] = pd.to_numeric(df['length'], errors='coerce').fillna(0).astype(int)
label_col = 'label' if 'label' in df.columns else 'classification'
if label_col in df.columns:
    df[label_col] = df[label_col].apply(lambda x: 0 if x in ['Normal', 0] else 1)
else:
    df['label'] = 0  # Create if missing
    print("Warning: No label/classification column found, created with default 0.")

# Step 10: Address outliers
df['outlier_flag'] = df.apply(
    lambda row: 'Yes' if ('URL' in df.columns and get_host(row['URL']) != normal_host) or row.get('Method', '') == 'PUT' else 'No',
    axis=1
)

# Save
df.to_csv('data/fully_cleaned_dataset.csv', index=False)
print(f"Cleaned dataset saved. Original rows: {len(pd.read_csv('data/csic_database.csv'))}, After cleaning: {len(df)}")

Original columns: ['Unnamed: 0', 'Method', 'User-Agent', 'Pragma', 'Cache-Control', 'Accept', 'Accept-encoding', 'Accept-charset', 'language', 'host', 'cookie', 'content-type', 'connection', 'lenght', 'content', 'classification', 'URL']
Dropped columns with >70.0% missing values: ['content-type', 'lenght', 'content']
Cleaned dataset saved. Original rows: 61065, After cleaning: 61065
